In [1]:
#import necessary libraries
import sempy.fabric as fabric
import pandas as pd
import numpy as np
import re
from notebookutils import mssparkutils
from datetime import date,datetime

#import sempy_labs as labs
#from sempy_labs import directlake
#from sempy_labs.tom import connect_semantic_model

###########################################################################################################
### Set Parameters
###########################################################################################################
#Define lakehouse and schema names
workspace_name = "amaser_dp700" #this is the workspace name where the notebook runs, and where the lakehouse lives
lakehouse_name = "test_fit" #this is the name of the lakehouse, in the event that it needs to becreated by this notebook
schema_name = "fitschema"  #This is the name of the schema.  If you want a schema-enabled lakehouse, you MUST create the lakehouse ahead of time, by hand


StatementMeta(, 750da3d7-2893-40ef-96f1-7e833797ea30, 3, Finished, Available, Finished)

In [2]:
#set default values
lakehouse_description = "Fabric Inventory Tool Lakehouse"  # this is only used if the notebook creates a new lakehouse
full_lakehouse_name = f'{workspace_name}.{lakehouse_name}.{schema_name}'



StatementMeta(, 750da3d7-2893-40ef-96f1-7e833797ea30, 4, Finished, Available, Finished)

In [3]:
###################################################################################################
### DEFINE FUNCTIONS
###################################################################################################
def logActivity(table_name,message):
    logquery = f'INSERT INTO {full_lakehouse_name}.inventorylog (timeval,workspace_id,Message) VALUES (CURRENT_TIMESTAMP(),"{table_name}","{message}")'
    #print(f'log query={logquery}')
    #print(logquery)
    spark.sql(logquery)

StatementMeta(, 750da3d7-2893-40ef-96f1-7e833797ea30, 5, Finished, Available, Finished)

In [4]:

# check if lakehouse exists.  If not, create a lakehouse 
try:
    lakehouse_object = mssparkutils.lakehouse.get(lakehouse_name)
    lakehouse_id=lakehouse_object.id
    print("Lakehouse exists")
except Exception as e:
        print ("Lakehouse does not exist.  Attempting to create lakehouse...")
        try:
            fabric.create_lakehouse(display_name=lakehouse_name,description=lakehouse_description,workspace=workspace_name)
            lakehouse_object = mssparkutils.lakehouse.get(lakehouse_name)
            print("Lakehouse created without schema enabling")
            lakehouse_id=lakehouse_object.id
            schema_name = "dbo"
        except Exception as e:
            print('Unable to create lakehouse')
            print(e)
    
#check if schema exists (if schema-enabled)
if schema_name == "dbo":
    print ("Default schema.  No check performed")
else:
    schema_sql  = f"CREATE SCHEMA IF NOT EXISTS {workspace_name}.{lakehouse_name}.{schema_name}"
    spark.sql(schema_sql)
    

StatementMeta(, 750da3d7-2893-40ef-96f1-7e833797ea30, 6, Finished, Available, Finished)

Lakehouse exists


In [5]:
# create tables
#create the log table. 
logquery = f'CREATE TABLE IF NOT EXISTS {full_lakehouse_name}.inventorylog (timeval TIMESTAMP, workspace_id STRING, Message STRING) USING DELTA'
#print(logquery)
spark.sql(logquery)
print("Log Table Created")
workspacequery = f'CREATE TABLE IF NOT EXISTS {full_lakehouse_name}.workspaces (timeval DATE, workspace_name STRING,workspace_id STRING,is_read_only BOOLEAN, is_dedicated_capacity BOOLEAN, capacity_id STRING,type STRING)'
spark.sql(workspacequery)
print("Workspace Table created")
itemsquery = f"""
    CREATE TABLE IF NOT EXISTS 
    {full_lakehouse_name}.fabricitems (timeval DATE, item_name STRING, item_id STRING,Description STRING, item_type STRING, workspace_id STRING,workspace_name STRING)
    """
spark.sql(itemsquery)
print("Items table created")

StatementMeta(, 750da3d7-2893-40ef-96f1-7e833797ea30, 7, Finished, Available, Finished)

Log Table Created
Workspace Table created
Items table created
